# How many datasets are enough?

As explained in the paper, given a desired generalizability $\alpha^*$ and a similarity threshold between rankings $\delta^*$, we show how to estimate the number of datasets required to obtain generalizable results. 
The procedure goes as follows: 
1. Load the experimental results. 
2. Query the results for a combination of design and held-constant factors.
3. Sample without repetition $N$ levels of the allowed-to-vary factor (`dataset`) and query the results accordingly. 
4. Rank the alternatives according to the target column. 
5. Estimate the $\alpha^*$-quantile of MMD for all $n \leq N$, call it $\varepsilon^{\alpha^*}_n$.
6. Fit a linear model $\log(n) = \beta_1 \log(\varepsilon^{\alpha^*}_n) + \beta_0$.
7. Estimate $n^* = \exp(\beta_1 \log(\varepsilon(\delta^*)) + \beta_0)$, where $\varepsilon(\delta^*)$ is a threshold on MMD.
We perform most of these operations within the main loop, iterating over the combinations of design factors. 

We can investigate the behavior of generalizability and $n^*$ for different $\alpha^*$ and $\delta^*$ by changing them in `config.yaml` and re-running the notebook. 
We analyze this in `demo2_plots.ipynb`. 

## Imports and configuration

In [1]:
import numpy as np
import pandas as pd
import yaml

from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from tqdm.auto import tqdm

from genexpy import lower_bounds as gu
from genexpy import kernels as ku
from genexpy import rankings_utils as ru
from genexpy import mmd as mmd

Next, we load the parameters from the configuration file. 

In [2]:
with open("config.yaml", 'r') as file:
    config = yaml.safe_load(file)

OUTPUT_DIR = Path(config['paths']['output_dir'])
FIGURES_DIR = Path(config['paths']['figures_dir'])

SEED = config['parameters']['seed']
RNG = np.random.default_rng(SEED)
ALPHA = config['parameters']['alpha']
LR_CONFIDENCE = config['parameters']['lr_confidence']
CI_LOWER = (1 - LR_CONFIDENCE) / 2
CI_UPPER = LR_CONFIDENCE + CI_LOWER

DATASET = Path(config['data']['dataset_path'])
EXPERIMENTAL_FACTORS = config['data']['experimental_factors']
TARGET = config['data']['target']
ALTERNATIVES = config['data']['alternatives']

SAMPLE_SIZE = config['sampling']['sample_size']
DISJOINT = config['sampling']['disjoint']
REPLACE = config['sampling']['replace']

Now we initialize the kernels, storing them in a dictionary together with heir parameters and the specified $\delta^*$ (one for each kernel). 

In [3]:
KERNELS = {}
for kernel_config in config['kernels']:
    kernel_func = getattr(ku, kernel_config['kernel'], None)
    
    if kernel_func:
        delta = kernel_config['delta']  # to get epsilon
        match kernel_config['kernel']:
            case "mallows_kernel":
                eps = np.sqrt(2 * (1 - np.exp(-delta)))  # assumes nu = 1/binom(n, 2)
            case "jaccard_kernel":
                eps = np.sqrt(2 * (1 - (1-delta)))
            case "borda_kernel":
                eps = np.sqrt(2 * (1 - np.exp(-delta)))   # assumes nu = 1/n
            case _ :
                raise ValueError(f"The kernel {kernel_config['kernel']} must be either the Jaccard, Mallows, or Borda kernel.")  

        for param_key, param_values in kernel_config['params'].items():
            if isinstance(param_values, list):
                for value in param_values:
                    params = {param_key: value}
                    kernel_name = f"{kernel_config['kernel']}_{param_key}_{value}"
                    KERNELS[kernel_name] = (kernel_func, params, eps, delta)
            else:
                params = {param_key: param_values}
                kernel_name = f"{kernel_config['kernel']}_{param_key}_{param_values}"
                KERNELS[kernel_name] = (kernel_func, params, eps, delta)
    else:
        print(f"Kernel function '{kernel_config['kernel']}' not found in module 'kernels'.")

We now load the dataset of results and perform some preliminary checks.

In [4]:
if DATASET.suffix == '.parquet':
    df = pd.read_parquet(DATASET).drop(columns=["time"])
elif DATASET.suffix == '.csv':
    df = pd.read_csv(DATASET).drop(columns=["time"])
else:
    raise Exception("Please use a Parquet or CSV file as the format of your data")

# Check whether exactly one of the experimental factors is None 
assert sum(value is None for value in EXPERIMENTAL_FACTORS.values()) == 1, "Exactly one experimental factor must be set to null in config.yaml."

# Check whether the factors listed in the config coincide with the columns of df
columns_to_check = set(EXPERIMENTAL_FACTORS.keys()).union({TARGET, ALTERNATIVES})
if not_in_df := columns_to_check - set(df.columns):
    raise ValueError(f"The following columns are missing from the dataframe: {not_in_df}")
if not_in_config:= set(df.columns) - columns_to_check:
    raise ValueError(f"The following columns in the dataframe are not required: {not_in_config}")

Finally, we query the dataset for the specified levels of the held-constant factors and get all the combinations of levels of design factors. 

In [5]:
# query fot the held-constant factors
try:
    query_string = " and ".join(f"{factor} == '{lvl}'" if isinstance(lvl, str) else f"{factor} == {lvl}"
                                for factor, lvl in EXPERIMENTAL_FACTORS.items()
                                if lvl not in [None, "_all"])
    df = df.query(query_string)
except ValueError:
    pass

# get the combinations of levels of allowed-to-vary-factors
try:
    groups = df.groupby([factor for factor, lvl in EXPERIMENTAL_FACTORS.items() if lvl == "_all"]).groups
except ValueError:
    groups = {"None": df.index}

## Utility functions

We now define all the functions we will need to estimate $n^*$ in the main loop. 

We start by creating the necessary directories.

In [6]:
def create_experiment_directory(kernel_name, factors, delta):
    exp0_dir = OUTPUT_DIR / "_".join([f"{key}={value}" for key, value in factors.items() if value is not None])
    exp1_dir = exp0_dir / f"{kernel_name}"
    exp21_dir = exp1_dir / f"nstar_N_ALPHA={ALPHA}_delta={delta}_ci={LR_CONFIDENCE}"
    exp21_dir.mkdir(parents=True, exist_ok=True)
    exp22_dir = exp1_dir / "computed_generalizability"
    exp22_dir.mkdir(parents=True, exist_ok=True)
    exp23_dir = exp1_dir / "computed_quantiles"
    exp23_dir.mkdir(parents=True, exist_ok=True)
    return exp21_dir, exp22_dir, exp23_dir

Then, we sample some levels from allowed-to-vary factor, query the dataset of results, and compute the rankings.

In [7]:
def sample_ecs(ec_pool, sample_size):
    assert sample_size <= len(ec_pool), f"Sample size {sample_size} is larger than |ec_pool| = {len(ec_pool)}"
    return RNG.choice(ec_pool, sample_size, replace=False)

def compute_rankings(ecs, rank_matrix):
    rm_ = rank_matrix.loc[:, ecs]
    na, nv = rm_.shape
    
    # Generate rankings from the data
    rankings = ru.SampleAM.from_rank_function_dataframe(rm_)
    
    return rankings, nv

On the rankings, we compute the variance and a lower bound on generalizability we build from [1] and then compute the Maximum ean Discrepancy (MMD) for pairs of samples of varying sizes $n$. 
We will use it to estimate the $\alpha^*$-quantile $\varepsilon^{\alpha^*}_n$.

In [8]:
def sample_mmd(rankings, nv, kernel, kernelargs):
    mmds = {
        n: mmd.subsample_mmd_distribution(
            rankings, subsample_size=n, rep=100, use_rv=True, use_key=False,
            seed=SEED, disjoint=DISJOINT, replace=REPLACE, kernel=kernel, **kernelargs
        )
        for n in range(2, nv // 2 + 1)
    }
    return mmds

With the samples of MMD, we calculate the generalizability and $\alpha^*$-quantile for each $n$.

In [9]:
def create_generalizability_dataframe(mmds, logepss):
    ys = {n: [mmd.generalizability(mmde, np.exp(logeps)) for logeps in logepss] for n, mmde in mmds.items()}
    dfy = pd.DataFrame(ys, index=logepss).reset_index().melt(id_vars='index', var_name='n', value_name='generalizability')
    dfy.rename(columns={'index': 'log(eps)'}, inplace=True)
    dfy['n'] = dfy['n'].astype(int)
    return dfy

def create_quantiles_dataframe(mmds):
    qs = {n: np.log(np.quantile(mmde, ALPHA)) for n, mmde in mmds.items()}
    dfq = pd.DataFrame(list(qs.items()), columns=['n', 'log(eps)'])
    dfq['log(n)'] = np.log(dfq['n'])
    return dfq

Finally, we fit linear regression $\log(n) = \beta_1\log(\varepsilon^{\alpha^*}_n) = \beta_0$ and use it to predit $n^*$ with a leave-one-out confidence interval.

In [10]:
def perform_linear_regression_with_cv(dfq):
    # Extracting features and target from DataFrame
    X = dfq[['log(eps)']].values
    y = dfq[['log(n)']].values

    cv = KFold(n_splits=len(y))
    residuals, linear_predictors = [], []

    for train_index, test_index in cv.split(X):
        lr = LinearRegression().fit(X[train_index], y[train_index])
        predicted = lr.predict(X[test_index])
        residuals.extend(y[test_index] - predicted)

        linear_predictors.append(lr)

    return linear_predictors, residuals

def predict_nstar(logepss, linear_predictors, dfq, eps):
    X = dfq[['log(eps)']].values
    y = dfq[['log(n)']].values

    ns_pred_cv = [np.exp(lr.predict(logepss.reshape(-1, 1)).reshape(-1)) for lr in linear_predictors]
    ns_pred = np.exp(LinearRegression().fit(X, y).predict(logepss.reshape(-1, 1)).reshape(-1))
    nstar_cv = [pred[np.argmax(logepss > np.log(eps))] for pred in ns_pred_cv if not np.all(pred == 0)]
    nstar = ns_pred[np.argmax(logepss > np.log(eps))]
    nstar_lower, nstar_upper = np.quantile(nstar_cv, [CI_LOWER, CI_UPPER])
    return ns_pred, ns_pred_cv, nstar, nstar_lower, nstar_upper

## Main loop

We are finally ready to run the main loop on the combinations of design factors.  

In [11]:
for fixed_levels, idxs in tqdm(list(groups.items()), position=0, desc="Configurations", leave=True):
    
    # Query the results for the fixed-levels
    idf = df.loc[idxs].reset_index(drop=True)
    if idf.empty:
        continue

    # Current levels of design and held-constant factor
    factors_dict = {factor: lvl
                    for factor, lvl in EXPERIMENTAL_FACTORS.items()
                    if lvl not in [None, "_all"]}
    factors_dict.update({factor: idf[factor].unique()[0] for factor, lvl in EXPERIMENTAL_FACTORS.items()
                         if lvl == "_all"})

    # Rank the alternatives  
    rank_matrix = ru.get_rankings_from_df(idf, factors=list(EXPERIMENTAL_FACTORS.keys()), 
                                            alternatives=ALTERNATIVES,
                                            target=TARGET,
                                            lower_is_better=False, impute_missing=True)
    # Impute the missing values
    rank_matrix = rank_matrix.fillna(rank_matrix.max())

    # Get the pool of experimental conditions to sample from (combinations of allowed-to-vary factors)
    atv_factor = next((key for key, value in EXPERIMENTAL_FACTORS.items() if value is None), None)
    ec_pool = idf[atv_factor].unique()
    ecs = np.array([])

    # Loop over the kernels
    for kernelname, (kernel, kernelargs, epsstar, deltastar) in KERNELS.items():
        
        # Update the parameters of the Borda kernel with the index of the alternative in the dataframe of rankings (might vary)
        if "borda" in kernelname:
            kernelargs.update({"idx": rank_matrix.index.get_loc(kernelargs["alternative"])})
        
        # Iteratively sample from ec_pool increasing the sample size at every iteration
        # The sampled experimental conditions are used to approximate the distribution of true results
        nstar_dir, gen_dir, quant_dir = create_experiment_directory(kernelname, factors_dict, epsstar)
        out = []
        # for i in tqdm(range(len(ec_pool) // SAMPLE_SIZE), desc=f'Using {kernelname}', leave=False):
        for i in range(len(ec_pool) // SAMPLE_SIZE):

            if (i+1)*SAMPLE_SIZE > len(ec_pool):
                break

            # Sample new experimental conditions from ec_pool and get their rankings
            ecs = sample_ecs(ec_pool, (i+1)*SAMPLE_SIZE)
            rankings, nv = compute_rankings(ecs, rank_matrix)

            # Compute the variance of the experimental results and get the lower bound (conservative estimate) on the distance between the sampled mean embedding and the true mean embedding in the RKHS
            variance = ku.var(rankings, use_rv=True, kernel=kernel, **kernelargs)
            var_lower_bound = gu.sample_mean_embedding_lowerbound(eps=epsstar, n=len(ecs), kbar=1, v=variance)

            # We do not need to compute dfy and dfq again if we have already computed them for another (alphastar, deltastar)
            if f"dfy_{len(ecs)}" in [x.stem for x in gen_dir.glob("*.parquet")] and f"dfmmd_{len(ecs)}" in [x.stem for x in quant_dir.glob("*.parquet")]:
                try:
                    dfy = pd.read_parquet(gen_dir / f"dfy_{len(ecs)}.parquet")
                    dfmmd = pd.read_parquet(quant_dir / f"dfmmd_{len(ecs)}.parquet")

                    dfq = pd.DataFrame(dfmmd.groupby("n")["eps"].quantile(ALPHA)).reset_index()
                    dfq["log(eps)"] = np.log(dfq["eps"])
                    dfq["log(n)"] = np.log(dfq["n"])

                    logepss = dfy["log(eps)"].unique()
                except Exception as e:
                    print("Exception thrown for the following combination of experimental conditions: ", factors_dict)
                    raise e
            else:
                # Sample the distribution of MMD for varying sizes  
                mmds = sample_mmd(rankings, nv, kernel=kernel, kernelargs=kernelargs)
                dfmmd = pd.DataFrame(mmds).melt(var_name="n", value_name="eps")

                # Compute generalizability and quantiles
                logepss = np.linspace(np.log(epsstar) - 0.1, np.log(max(np.quantile(mmde, ALPHA) for mmde in mmds.values())) + 0.1, 1000)  # the bounds are chosen to optimize 
                dfy = create_generalizability_dataframe(mmds, logepss)
                dfq = create_quantiles_dataframe(mmds)

            # Linear regression with cross-validation
            linear_predictors, residuals = perform_linear_regression_with_cv(dfq)

            # Predictions
            ns_pred, ns_pred_cv, nstar, nstar_lower, nstar_upper = predict_nstar(logepss, linear_predictors, dfq, epsstar)

            # Store results
            result_dict = {
                "kernel": kernelname,
                "alpha": ALPHA,
                "eps": epsstar,
                "delta": deltastar,
                "disjoint": DISJOINT,
                "replace": REPLACE,
                "N": len(ecs),
                "nstar": nstar,
                "nstar_lower": nstar_lower,
                "nstar_upper": nstar_upper,
                "variance": variance,
                "var_lower_bound": var_lower_bound,
            }
            result_dict.update(factors_dict)
            out.append(result_dict)

            dfy.to_parquet(gen_dir / f"dfy_{len(ecs)}.parquet")
            # dfq.to_parquet(quant_dir / f"dfq_{len(ecs)}_{ALPHA}.parquet")  # not needed 
            dfmmd.to_parquet(quant_dir / f"dfmmd_{len(ecs)}.parquet")

        # Store nstar predictions
        out = pd.DataFrame(out)
        out.to_parquet(nstar_dir / "nstar.parquet")

Configurations:   0%|          | 0/20 [00:00<?, ?it/s]

## References

[1] Wolfer, Geoffrey, and Pierre Alquier. *Variance-aware estimation of kernel mean embedding.* arXiv preprint arXiv:2210.06672 (2022).